In [46]:
import numpy as np
import pandas as pd
Series = pd.Series
DataFrame = pd.DataFrame
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
doc = pd.read_csv("US_Accidents_March23.csv")

In [3]:
doc.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


In [4]:
doc.columns

Index(['ID', 'Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat',
       'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description',
       'Street', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [5]:
doc = doc.reindex(columns=["ID", "Start_Time", "City", "State", "Temperature(F)", "Visibility(mi)", "Wind_Speed(mph)", "Precipitation(in)", "Weather_Condition"])
doc.head(100)

,ID,Start_Time,City,State,Temperature(F),Visibility(mi),Wind_Speed(mph),Precipitation(in),Weather_Condition
0,A-1,2016-02-08 05:46:00,Dayton,OH,36.9,10.0,NaN,0.02,Light Rain
1,A-2,2016-02-08 06:07:59,Reynoldsburg,OH,37.9,10.0,NaN,0.00,Light Rain
2,A-3,2016-02-08 06:49:27,Williamsburg,OH,36.0,10.0,3.5,NaN,Overcast
3,A-4,2016-02-08 07:23:34,Dayton,OH,35.1,9.0,4.6,NaN,Mostly Cloudy
4,A-5,2016-02-08 07:39:07,Dayton,OH,36.0,6.0,3.5,NaN,Mostly Cloudy
...,...,...,...,...,...,...,...,...,...
95,A-96,2016-02-11 07:28:16,Pataskala,OH,7.0,10.0,NaN,NaN,Overcast
96,A-97,2016-02-11 07:29:06,Cleveland,OH,15.1,10.0,18.4,NaN,Mostly Cloudy
97,A-98,2016-02-11 07:53:55,Dayton,OH,9.0,9.0,NaN,NaN,Clear
98,A-99,2016-02-11 08:07:02,Dayton,OH,7.5,10.0,4.6,NaN,Scattered Clouds


In [6]:
doc = doc.dropna(subset=["Weather_Condition"])
len(doc)

7554935

In [7]:
cities = ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix", "Philadelphia", "San Antonio", "San Diego", "Dallas", "Austin", "Jacksonville"]

In [8]:
# For experimenting with a data set small enough that operations run
# instantly. Not for analysis: just data manipulation.
doc_head = doc.head(10000).copy()
doc_head = doc_head.reindex(columns=["Start_Time", "City", "Weather_Condition"])

In [9]:
doc = doc[doc["City"].isin(cities)]
doc.head()

,ID,Start_Time,City,State,Temperature(F),Visibility(mi),Wind_Speed(mph),Precipitation(in),Weather_Condition
42866,A-42867,2016-06-21 10:46:30,Los Angeles,CA,82.9,10.0,4.6,NaN,Clear
42867,A-42868,2016-06-21 10:49:21,Los Angeles,CA,82.9,10.0,4.6,NaN,Clear
42881,A-42882,2016-06-21 10:51:45,Los Angeles,CA,82.9,10.0,4.6,NaN,Clear
42883,A-42884,2016-06-21 10:56:24,Los Angeles,CA,82.9,10.0,4.6,NaN,Clear
42886,A-42887,2016-06-21 10:57:39,San Diego,CA,69.1,9.0,8.1,NaN,Scattered Clouds


In [10]:
DataFrame(doc_head.groupby(["City", "Weather_Condition"])).iloc[0, 1]

,Start_Time,City,Weather_Condition
1465,2016-06-27 18:35:11,Acampo,Clear
1574,2016-06-28 16:45:10,Acampo,Clear
1967,2016-07-03 08:09:37,Acampo,Clear
2187,2016-07-05 17:09:47,Acampo,Clear
2652,2016-07-10 11:15:17,Acampo,Clear
2953,2016-07-13 18:51:38,Acampo,Clear
3054,2016-07-14 16:49:45,Acampo,Clear
4185,2016-07-25 19:53:25,Acampo,Clear
4314,2016-07-26 15:49:08,Acampo,Clear
6851,2016-12-12 15:33:24,Acampo,Clear


In [11]:
def previous_hour_mark(date_str):
    date, time = date_str.split()
    time = time[:2]
    return date + " " + time

In [12]:
doc["Start_Time"] = doc["Start_Time"].apply(previous_hour_mark)
doc

,ID,Start_Time,City,State,Temperature(F),Visibility(mi),Wind_Speed(mph),Precipitation(in),Weather_Condition
42866,A-42867,2016-06-21 10,Los Angeles,CA,82.9,10.0,4.6,NaN,Clear
42867,A-42868,2016-06-21 10,Los Angeles,CA,82.9,10.0,4.6,NaN,Clear
42881,A-42882,2016-06-21 10,Los Angeles,CA,82.9,10.0,4.6,NaN,Clear
42883,A-42884,2016-06-21 10,Los Angeles,CA,82.9,10.0,4.6,NaN,Clear
42886,A-42887,2016-06-21 10,San Diego,CA,69.1,9.0,8.1,NaN,Scattered Clouds
...,...,...,...,...,...,...,...,...,...
7728364,A-7777732,2019-08-23 13,Los Angeles,CA,82.0,9.0,0.0,0.0,Fair
7728374,A-7777742,2019-08-23 15,San Diego,CA,74.0,10.0,7.0,0.0,Fair
7728376,A-7777744,2019-08-23 15,Los Angeles,CA,74.0,10.0,8.0,0.0,Fair
7728377,A-7777745,2019-08-23 16,San Diego,CA,75.0,10.0,7.0,0.0,Fair


In [13]:
def most_common(df, column="Weather_Condition"):
    return Series([df[column].mode()[0], len(df[column])]) 

In [14]:
aggregated = doc.groupby(["Start_Time", "City"]).apply(most_common)
aggregated

0  1
Start_Time    City                         
2016-03-22 19 Los Angeles          Clear  1
2016-03-22 20 Los Angeles          Clear  2
              San Diego            Clear  1
2016-03-22 22 Los Angeles          Clear  2
2016-03-23 00 San Diego    Partly Cloudy  1
...                                  ... ..
2023-03-31 21 Chicago             Cloudy  3
              Phoenix      Mostly Cloudy  1
2023-03-31 22 Chicago      Mostly Cloudy  1
              Dallas                Fair  2
2023-03-31 23 Dallas                Fair  1

[270364 rows x 2 columns]

In [31]:
ag_head = aggregated.head(10_000)
ag_head

0  1
Start_Time    City                         
2016-03-22 19 Los Angeles          Clear  1
2016-03-22 20 Los Angeles          Clear  2
              San Diego            Clear  1
2016-03-22 22 Los Angeles          Clear  2
2016-03-23 00 San Diego    Partly Cloudy  1
...                                  ... ..
2016-09-04 22 Los Angeles          Clear  2
2016-09-05 01 Los Angeles       Overcast  1
2016-09-05 02 Los Angeles       Overcast  3
2016-09-05 03 Los Angeles  Partly Cloudy  1
              San Antonio  Mostly Cloudy  1

[10000 rows x 2 columns]

In [34]:
ag_head.iloc[2]

0    Clear
1        1
Name: (2016-03-22 20, San Diego), dtype: object

In [41]:
len(ag_head)

10000

In [44]:
[ag_head.iloc[0][1], ag_head.iloc[0].name[1], ag_head.iloc[0].name[0], ag_head.iloc[0][0]]
data = {
    "Crashes": [ag_head.iloc[i][1] for i in range(len(ag_head))],
    "City": [ag_head.iloc[i].name[1] for i in range(len(ag_head))],
    "Hour": [ag_head.iloc[i].name[0][-2:] for i in range(len(ag_head))],
    "Weather": [ag_head.iloc[i][0] for i in range(len(ag_head))],
}
flat_head = DataFrame(data)
flat_head

,Crashes,City,Hour,Weather
0,1,Los Angeles,19,Clear
1,2,Los Angeles,20,Clear
2,1,San Diego,20,Clear
3,2,Los Angeles,22,Clear
4,1,San Diego,00,Partly Cloudy
...,...,...,...,...
9995,2,Los Angeles,22,Clear
9996,1,Los Angeles,01,Overcast
9997,3,Los Angeles,02,Overcast
9998,1,Los Angeles,03,Partly Cloudy


In [45]:
# There is surely a better way to do this
data = {
    "Crashes": [aggregated.iloc[i][1] for i in range(len(aggregated))],
    "City": [aggregated.iloc[i].name[1] for i in range(len(aggregated))],
    "Hour": [aggregated.iloc[i].name[0][-2:] for i in range(len(aggregated))],
    "Weather": [aggregated.iloc[i][0] for i in range(len(aggregated))],
}
flat = DataFrame(data)
flat

,Crashes,City,Hour,Weather
0,1,Los Angeles,19,Clear
1,2,Los Angeles,20,Clear
2,1,San Diego,20,Clear
3,2,Los Angeles,22,Clear
4,1,San Diego,00,Partly Cloudy
...,...,...,...,...
270359,3,Chicago,21,Cloudy
270360,1,Phoenix,21,Mostly Cloudy
270361,1,Chicago,22,Mostly Cloudy
270362,2,Dallas,22,Fair


In [121]:
train, test = train_test_split(flat.copy(), 
                               shuffle=True,
                               random_state=614,
                               test_size=.2)

In [124]:
train.sample(5)

,Crashes,City,Hour,Weather
17286,3,San Diego,09,Mostly Cloudy
156179,1,Houston,04,Partly Cloudy
86998,6,Austin,16,Partly Cloudy
163361,3,Los Angeles,02,Fair
98010,1,Austin,21,Scattered Clouds


In [56]:
# free up some memory here
del doc
del aggregated

In [125]:
train.loc[:, pd.get_dummies(train["City"]).columns] = pd.get_dummies(train['City'], dtype=int).copy()
train

,Crashes,City,Hour,Weather,Austin,Chicago,Dallas,Houston,Jacksonville,Los Angeles,New York,Philadelphia,Phoenix,San Antonio,San Diego
60103,3,Phoenix,18,Clear,0,0,0,0,0,0,0,0,1,0,0
165703,1,Dallas,02,Cloudy,0,0,1,0,0,0,0,0,0,0,0
249617,5,San Diego,07,Fair,0,0,0,0,0,0,0,0,0,0,1
41012,3,Chicago,14,Mostly Cloudy,0,1,0,0,0,0,0,0,0,0,0
117208,1,Los Angeles,16,Cloudy,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144783,1,San Antonio,18,Light Rain,0,0,0,0,0,0,0,0,0,1,0
61434,8,Austin,08,Overcast,1,0,0,0,0,0,0,0,0,0,0
242204,6,Dallas,06,Fair,0,0,1,0,0,0,0,0,0,0,0
71451,1,Dallas,15,Clear,0,0,1,0,0,0,0,0,0,0,0


In [126]:
train.loc[:, pd.get_dummies(train["Weather"]).columns] = pd.get_dummies(train['Weather'], dtype=int).copy()
train

,Crashes,City,Hour,Weather,Austin,Chicago,Dallas,Houston,Jacksonville,Los Angeles,...,Thunder,Thunder / Windy,Thunder / Wintry Mix,Thunder in the Vicinity,Thunderstorm,Thunderstorms and Rain,Volcanic Ash,Widespread Dust,Wintry Mix,Wintry Mix / Windy
60103,3,Phoenix,18,Clear,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
165703,1,Dallas,02,Cloudy,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
249617,5,San Diego,07,Fair,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41012,3,Chicago,14,Mostly Cloudy,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
117208,1,Los Angeles,16,Cloudy,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144783,1,San Antonio,18,Light Rain,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61434,8,Austin,08,Overcast,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
242204,6,Dallas,06,Fair,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71451,1,Dallas,15,Clear,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [127]:
train.loc[:, pd.get_dummies(train["Hour"]).columns] = pd.get_dummies(train['Hour'], dtype=int).copy()
train

,Crashes,City,Hour,Weather,Austin,Chicago,Dallas,Houston,Jacksonville,Los Angeles,...,14,15,16,17,18,19,20,21,22,23
60103,3,Phoenix,18,Clear,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
165703,1,Dallas,02,Cloudy,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
249617,5,San Diego,07,Fair,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41012,3,Chicago,14,Mostly Cloudy,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
117208,1,Los Angeles,16,Cloudy,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144783,1,San Antonio,18,Light Rain,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
61434,8,Austin,08,Overcast,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
242204,6,Dallas,06,Fair,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71451,1,Dallas,15,Clear,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [145]:
# With no interaction terms
no_interaction_lr = LinearRegression(copy_X = True, positive=True, fit_intercept=False)

# Build the columns we use since there are many.

categories = list(train["Hour"].unique()) + list(train["Weather"].unique()) + list(train["City"].unique())

no_interaction_lr.fit(train[categories].values,
                      train['Crashes'].values)

LinearRegression(fit_intercept=False, positive=True)

In [172]:
assert len(categories) == 111

In [146]:
no_interaction_lr.coef_

array([1.40884151e+00, 9.45262351e-02, 2.05534481e+00, 1.30885459e+00,
       1.80931529e+00, 4.61812675e-01, 1.12700940e+00, 5.85259779e-01,
       1.61731790e+00, 1.11730131e+00, 1.04783131e+00, 8.50129540e-01,
       2.30495462e-01, 1.34177742e+00, 1.57393241e-01, 1.89229197e-01,
       1.93125511e+00, 1.37344051e+00, 2.30459040e+00, 1.27992113e+00,
       2.61416194e-01, 0.00000000e+00, 1.85092247e-01, 1.33404310e-01,
       2.49294721e-01, 5.20358911e-01, 7.42167745e-01, 3.46876579e-01,
       1.84178839e-01, 3.49563595e-01, 3.40760213e-01, 7.85154951e-01,
       1.13079005e+00, 2.24514200e-01, 8.52741464e-02, 1.58115868e-01,
       1.18584571e+00, 7.86181510e-01, 7.04760497e-01, 9.11389841e-01,
       1.83256227e+00, 3.03007425e-01, 9.51334454e-01, 1.77759638e+00,
       7.45898031e-01, 6.22989893e-01, 2.92944106e+00, 1.13048381e+00,
       2.27309180e+00, 3.93217359e-01, 0.00000000e+00, 2.24927672e+00,
       0.00000000e+00, 1.11334694e+00, 9.57882368e-01, 3.67165600e-01,
      

In [149]:
no_interaction_lr.predict(train[categories].values[:10])

array([2.26094268, 2.54879067, 3.48462453, 1.82112646, 4.3337926 ,
       1.07345495, 3.63236053, 3.80308267, 4.1510988 , 4.05514073])

In [148]:
no_interaction_lr.score(train[categories].values,
                      train['Crashes'].values)

0.1473255084360212

In [163]:
def encode(city, hour, weather):
    l = list(train[categories].columns)
    city_index = l.index(city)
    try:
        hour_index = l.index(hour)
    except ValueError:
        s = str(hour)
        if len(s) == 1:
            s = "0" + s
        hour_index = l.index(s)
    weather_index = l.index(weather)
    ans = [0 for _ in range(111)]
    ans[city_index] = 1
    ans[hour_index] = 1
    ans[weather_index] = 1
    return np.array(ans)
    

In [167]:
no_interaction_lr.predict([encode("Austin", "06", "Clear")])

array([3.21342184])

In [171]:
no_interaction_lr.predict([encode("Austin", 0, "Clear")])

array([2.02903766])